In [ ]:
pip install azure.storage.blob

In [ ]:
blob_service_client=""
connection_string = ""
da=""
source_container_name=""
destination_container_name=""
records_read=0
record_inserted=0
df=""
system_id = 0
trigged_by = dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()

In [ ]:
def check_seed_table():
    qury="select * from spark_catalog.bcp_control.tb_system_seed where Application_Name = 'unstructured engineering' and Source_DB = 'landing'"
    print(qury)
    systemdf= spark.sql(qury)
    if(systemdf.count()==0):
        raise Exception("Data not present")
    return systemdf

try:
    systemdf=check_seed_table()
    system_id=systemdf.collect()[0].System_Id
    source_container_name = systemdf.collect()[0].Source_DB
    destination_container_name = systemdf.collect()[0].Target_DB
    source_folder_name=systemdf.collect()[0].Source_Object_Name
    destination_folder_name=systemdf.collect()[0].Target_Object_Name
    display(systemdf)
except Exception as e:
    exp=str(e)
    my_json={"records_read":records_read,"record_inserted":record_inserted,"trigged_by":trigged_by,"exception":exp,"system_id":system_id}
    dbutils.notebook.exit(my_json)

print(str(system_id)+","+source_container_name+","+source_folder_name+","+destination_container_name+","+destination_folder_name)

In [ ]:
def get_audioFile_data():
    return spark.createDataFrame(dbutils.fs.ls("dbfs:/mnt/landing/AudioFiles/"))

def get_nameOfAudioFile(df):
    df.createOrReplaceTempView("audioFile")
    return spark.sql("select name from audioFile")

try:
    df=get_audioFile_data()
    if(df.count()== 0):
        exp="No new audio file is present"
        my_json={"records_read":records_read,"record_inserted":record_inserted,"trigged_by":trigged_by,"exception":exp,"system_id":system_id}
        dbutils.notebook.exit(my_json)
    da=get_nameOfAudioFile(df)
except Exception as e:
    exp=str(e)
    my_json={"records_read":records_read,"record_inserted":record_inserted,"trigged_by":trigged_by,"exception":exp,"system_id":system_id}
    dbutils.notebook.exit(my_json)



In [ ]:
from azure.storage.blob import BlobServiceClient, BlobType
def create_connection(conn_string):
    return BlobServiceClient.from_connection_string(conn_string)

try:
    blob_service_client = create_connection(connection_string)
except Exception as e:
    exp=str(e)
    my_json={"records_read":records_read,"record_inserted":record_inserted,"trigged_by":trigged_by,"exception":exp,"system_id":system_id}
    dbutils.notebook.exit(my_json)

In [ ]:

def read_audio(source_container_name,source_blob_name):
    return blob_service_client.get_blob_client(container=source_container_name, blob=source_blob_name,snapshot=None)


In [ ]:
def writing_audio(destination_blob_name,source_blob_client):
    destination_blob_client= blob_service_client.get_blob_client(container=destination_container_name, blob=destination_blob_name)
    copy_operation = destination_blob_client.start_copy_from_url(source_blob_client.url)
    copy_status = destination_blob_client.get_blob_properties().copy.status
    while copy_status == "pending":
        copy_status = destination_blob_client.get_blob_properties().copy.status
    if copy_status == "success":
        print ("Copied Blob URL:", destination_blob_client.url)
    else:
        print("Blob copy operation failed.")
    return copy_status
    

In [ ]:
data_collect = da.collect()
records_read = da.count()
flag=0
for row in data_collect:
    source_blob_name=str("AudioFiles/"+row["name"])
    destination_blob_name=str("AudioFiles/"+str(row["name"]))
    print(source_blob_name + "  "+destination_blob_name)
    try:
        source_blob_client = read_audio(source_container_name,source_blob_name)
        print("source_blob_client executed")
        status = writing_audio(destination_blob_name,source_blob_client)
        print( "status  executed"+ str(status))
        if(status != "success"):
            flag=1
            print("failed")
            break
    except Exception as e:
        exp=str(e)
        my_json={"records_read":records_read,"record_inserted":record_inserted,"trigged_by":trigged_by,"exception":exp,"system_id":system_id}
        dbutils.notebook.exit(my_json)

record_inserted=da.count()

In [ ]:
file_location =df.collect()
display(df)
if flag==0:
    for r in file_location:
        print(r)
        dbutils.fs.mv(r["path"],"dbfs:/mnt/landing/Archive/"+r["name"])

In [ ]:
my_json={"records_read":records_read,"record_inserted":record_inserted,"trigged_by":trigged_by,"exception":'',"system_id":system_id}
dbutils.notebook.exit(my_json)